In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import statistics
import sklearn
%matplotlib inline
from datetime import date
from scipy import stats as stats
from sklearn import preprocessing
from sklearn import model_selection
from datetime import datetime, timedelta
from tensorflow import keras as keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from numpy import asarray
from numpy import savetxt

In [2]:
df_all = pd.read_csv("/Users/alex/50varcomplete.csv")

In [5]:
#Make a table of each vital with its mask
HR = df_all[['HADM_ID','HeartRate', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'HeartRateMask']]
Temp = df_all[['HADM_ID','Temperature', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'TemperatureMask']]
WBC = df_all[['HADM_ID','White_Blood_Cell', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'White_Blood_CellMask']]
Glas = df_all[['HADM_ID','Glasgow_Coma', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'Glasgow_ComaMask']]
Biril = df_all[['HADM_ID','Birillium', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'BirilliumMask']]
BUN = df_all[['HADM_ID','BloodUreaNitrogen', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'BloodUreaNitrogenMask']]
Mech_Vent = df_all[['HADM_ID','Mech_Vent', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'Mech_VentMask']]
Albumin = df_all[['HADM_ID','Albumin', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'AlbuminMask']]
RBC = df_all[['HADM_ID','RBC', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'RBCMask']]
Hematocrit = df_all[['HADM_ID','Hematocrit', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'HematocritMask']]
Platelets = df_all[['HADM_ID','Platelets', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'PlateletsMask']]
Weight = df_all[['HADM_ID','Weight', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'WeightMask']]
pH = df_all[['HADM_ID','pH', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'pHMask']]
OxySat = df_all[['HADM_ID','OxySat', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'OxySatMask']]
ResRate = df_all[['HADM_ID','ResRate', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'ResRateMask']]


In [4]:
df_demo = df_all[['HADM_ID',
       'blood and blood-forming organs', 'circulatory', 'congenital',
       'digestive', 'endocrine, metabolic and immunity',
       'external causes of injury and supplemental classification',
       'genitourinary', 'ill-defined', 'infectious and parasitic',
       'injury and poisoning', 'mental', 'musculoskeletal', 'neoplasms',
       'nervous', 'perinatal period', 'pregnancy, childbirth, and puerperium',
       'respiratory', 'skin and subcutaneous tissue', 'ASIAN', 'BLACK',
       'HISPANIC', 'OTHER', 'UNKNOWNethnicity', 'WHITE', 'MARRIAGE ENDED',
       'MARRIED', 'SINGLE', 'UNKNOWNmarriage', 'CATHOLIC', 'CHRISTIAN OTHER',
       'JEWISH', 'NON-CHRISTIAN OTHER', 'PROTESTANT', 'UNKNOWNreligion',
       'GOVERNMENT', 'PRIVATE', 'SELF-PAY', 'CLINIC REFERRAL/PREMATURE',
       'EMERGENCY ROOM ADMIT', 'HMO REFERRAL/SICK',
       'PHYS REFERRAL/NORMAL DELI', 'TRANSFER FROM HOSP/EXTRAM',
       'TRANSFER FROM OTHER HEALT', 'TRANSFER FROM SKILLED NUR',
       'TRSF WITHIN THIS FACILITY', 'UNKNOWN', 'ELECTIVE', 'EMERGENCY',
       'URGENT', 'AGE BIN', 'Age Bin_2', 'Age Bin_3', 'Age Bin_4', 'Age Bin_5',
       'Age Bin_6']]

In [5]:
#Create a list to make columns for timeframe
names = []
for i in range(0, 336):
    j = str(i)
    names.append(j) 

In [ ]:
#Makes y for prediction
def makeanswer(data):
    length = 336
    hadmlist = data['HADM_ID'].unique()
    timeline = pd.DataFrame({'HADM_ID': hadmlist},columns = names)
    timeline['HADM_ID'] = hadmlist
    y = []
    for i in range(0, len(hadmlist)):
        df = data[data['HADM_ID'] == hadmlist[i]]
        lst = []
        counter = 0
        for j in range(1, (len(df)-1)):
            pos = int(df.iloc[j, 2])
            left = int(df.iloc[j, 3])
            if pos > 335:
                pass
            else:
                lst.append(left)
        y.append(lst)
    
    return y

In [ ]:
answers = makeanswer(HR)

In [ ]:
len(answers)

In [11]:
y = pd.DataFrame(answers)
y.to_csv(r'/Users/alex/LOSDatay.csv')

In [6]:
def makeset(data, value):
    length = 336
    hadmlist = data['HADM_ID'].unique()
    timeline = pd.DataFrame({'HADM_ID': hadmlist},columns = names)
    mask = pd.DataFrame({'HADM_ID': hadmlist},columns = names)
    timeline['HADM_ID'] = hadmlist
    mask['HADM_ID'] = hadmlist
    cols = []
    c = []
    y = []
    percent = 0
    for i in range(0, len(hadmlist)-1):
        if (i % 224 ) == 0:
            percent += 1
            print(str(percent) + '% done.') 
        df = data[data['HADM_ID'] == hadmlist[i]]
        lst = []
        counter = 0
        for j in range(1, (len(df)-1)):
            timein = int(df.iloc[j, 4])
            value = int(df.iloc[j, 1])
            pos = int(df.iloc[j, 2])
            left = int(df.iloc[j, 3])
            if pos > 335:
                pass
            else:
                timeline.iloc[i, pos] = value
                mask.iloc[i, pos] = 1
                lst.append(left)
            if timein < length:
                timeline.iloc[i, timein] = -1

        y.append(lst)
        
    timeline = timeline.iloc[:, 1:]
    mask = mask.iloc[:, 1:]
    mask = mask.fillna(0)
    timeline = timeline.ffill(axis = 1)
    timeline = timeline.fillna(value=value)
    
    return timeline, mask, y

In [7]:
def makechannel(data, name):
    channel = pd.DataFrame()
    for i in range(0, 336):
        j = str(i)
        mname = ('m' + name + j)
        tname = ('t'+ name + j)
        channel[tname] = data[0].iloc[:, i]
        channel[mname] = data[1].iloc[:,i]
    return channel

In [25]:
df_demo = pd.read_csv("/Users/alex/LOSDataDemo.csv")

In [26]:
hadmlist = HR['HADM_ID'].unique()
hadmlist = pd.DataFrame(hadmlist)
hadmlist['HADM_ID'] = hadmlist[0]

In [27]:
demodata = pd.merge(hadmlist, df_demo, on='HADM_ID')

In [28]:
demodata = demodata.drop_duplicates(subset = ['HADM_ID'])

In [29]:
dfmissing = pd.DataFrame({'HADM_ID': [107486, 130888, 136883, 141974, 189968], 
  'blood and blood-forming organs' : [0,0,0,0,0],
                     'circulatory' : [0,0,0,0,0],
                      'congenital' : [0,0,0,0,0],
                       'digestive' : [0,0,0,0,0],
'endocrine, metabolic and immunity' : [0,0,0,0,0],
'external causes of injury and supplemental classification' : [0,0,0,0,0],
                   'genitourinary' : [0,0,0,0,0],
                     'ill-defined' : [0,0,0,0,0],
        'infectious and parasitic' : [0,0,0,0,0],
            'injury and poisoning' : [0,0,0,0,0],
                          'mental' : [0,0,0,0,0],
                 'musculoskeletal' : [0,0,0,0,0],
                       'neoplasms' : [0,0,0,0,0],
                         'nervous' : [0,0,0,0,0],
                'perinatal period' : [0,0,0,0,0],
'pregnancy, childbirth, and puerperium' : [0,0,0,0,0],
                     'respiratory' : [0,0,0,0,0],
    'skin and subcutaneous tissue' : [0,0,0,0,0],
                           'ASIAN' : [0,0,0,0,0],
                           'BLACK' : [0,0,0,0,0],
                        'HISPANIC': [0,0,0,0,0],
                           'OTHER': [0,0,0,0,0],
                'UNKNOWNethnicity': [0,0,0,0,0],
                           'WHITE': [0,0,0,0,0],
                  'MARRIAGE ENDED': [0,0,0,0,0],
                         'MARRIED': [0,0,0,0,0],
                          'SINGLE': [0,0,0,0,0],
                 'UNKNOWNmarriage': [0,0,0,0,0],
                        'CATHOLIC': [0,0,0,0,0],
                 'CHRISTIAN OTHER': [0,0,0,0,0],
                          'JEWISH': [0,0,0,0,0],
             'NON-CHRISTIAN OTHER': [0,0,0,0,0],
                      'PROTESTANT': [0,0,0,0,0],
                 'UNKNOWNreligion': [0,0,0,0,0],
                      'GOVERNMENT': [0,0,0,0,0],
                         'PRIVATE': [0,0,0,0,0],
                        'SELF-PAY': [0,0,0,0,0],
       'CLINIC REFERRAL/PREMATURE': [0,0,0,0,0],
            'EMERGENCY ROOM ADMIT': [0,0,0,0,0],
               'HMO REFERRAL/SICK': [0,0,0,0,0],
       'PHYS REFERRAL/NORMAL DELI': [0,0,0,0,0],
       'TRANSFER FROM HOSP/EXTRAM': [0,0,0,0,0],
       'TRANSFER FROM OTHER HEALT': [0,0,0,0,0],
       'TRANSFER FROM SKILLED NUR': [0,0,0,0,0],
       'TRSF WITHIN THIS FACILITY': [0,0,0,0,0],
                         'UNKNOWN': [0,0,0,0,0],
                        'ELECTIVE': [0,0,0,0,0],
                       'EMERGENCY': [0,0,0,0,0],
                          'URGENT': [0,0,0,0,0],
                         'AGE BIN': [0,0,0,0,0],
                       'Age Bin_2': [0,0,0,0,0],
                       'Age Bin_3': [0,0,0,0,0],
                       'Age Bin_4': [0,0,0,0,0],
                       'Age Bin_5': [0,0,0,0,0],
                       'Age Bin_6': [0,0,0,0,0]})

In [30]:
demodata = demodata.fillna(0)

In [31]:
demodata = demodata.append(dfmissing)

In [32]:
demodata.shape

(22404, 59)

In [33]:
demodata.to_csv(r'/Users/alex/LOSDataDemo.csv')

In [27]:
demodata.to_csv(r'/Users/alex/LOSDataDemo.csv')

In [10]:
HR = makeset(HR, 86)
HRSave = makechannel(HR, 'HR')
HRSave.to_csv(r'/Users/alex/LOSDataHR.csv')

1% done.
2% done.
3% done.
4% done.
5% done.
6% done.
7% done.
8% done.
9% done.
10% done.
11% done.
12% done.
13% done.
14% done.
15% done.
16% done.
17% done.
18% done.
19% done.
20% done.
21% done.
22% done.
23% done.
24% done.
25% done.
26% done.
27% done.
28% done.
29% done.
30% done.
31% done.
32% done.
33% done.
34% done.
35% done.
36% done.
37% done.
38% done.
39% done.
40% done.
41% done.
42% done.
43% done.
44% done.
45% done.
46% done.
47% done.
48% done.
49% done.
50% done.
51% done.
52% done.
53% done.
54% done.
55% done.
56% done.
57% done.
58% done.
59% done.
60% done.
61% done.
62% done.
63% done.
64% done.
65% done.
66% done.
67% done.
68% done.
69% done.
70% done.
71% done.
72% done.
73% done.
74% done.
75% done.
76% done.
77% done.
78% done.
79% done.
80% done.
81% done.
82% done.
83% done.
84% done.
85% done.
86% done.
87% done.
88% done.
89% done.
90% done.
91% done.
92% done.
93% done.
94% done.
95% done.
96% done.
97% done.
98% done.
99% done.
100% done.


In [11]:
Albumin = makeset(Albumin, 4.4)
AlbuminSave = makechannel(Albumin, 'Albumin')
AlbuminSave.to_csv(r'/Users/alex/LOSDataAlbumin.csv')

1% done.
2% done.
3% done.
4% done.
5% done.
6% done.
7% done.
8% done.
9% done.
10% done.
11% done.
12% done.
13% done.
14% done.
15% done.
16% done.
17% done.
18% done.
19% done.
20% done.
21% done.
22% done.
23% done.
24% done.
25% done.
26% done.
27% done.
28% done.
29% done.
30% done.
31% done.
32% done.
33% done.
34% done.
35% done.
36% done.
37% done.
38% done.
39% done.
40% done.
41% done.
42% done.
43% done.
44% done.
45% done.
46% done.
47% done.
48% done.
49% done.
50% done.
51% done.
52% done.
53% done.
54% done.
55% done.
56% done.
57% done.
58% done.
59% done.
60% done.
61% done.
62% done.
63% done.
64% done.
65% done.
66% done.
67% done.
68% done.
69% done.
70% done.
71% done.
72% done.
73% done.
74% done.
75% done.
76% done.
77% done.
78% done.
79% done.
80% done.
81% done.
82% done.
83% done.
84% done.
85% done.
86% done.
87% done.
88% done.
89% done.
90% done.
91% done.
92% done.
93% done.
94% done.
95% done.
96% done.
97% done.
98% done.
99% done.
100% done.


In [12]:
RBC = makeset(RBC, 5)
RBCSave = makechannel(RBC, 'RBC')
RBCSave.to_csv(r'/Users/alex/LOSDataRBC.csv')

1% done.
2% done.
3% done.
4% done.
5% done.
6% done.
7% done.
8% done.
9% done.
10% done.
11% done.
12% done.
13% done.
14% done.
15% done.
16% done.
17% done.
18% done.
19% done.
20% done.
21% done.
22% done.
23% done.
24% done.
25% done.
26% done.
27% done.
28% done.
29% done.
30% done.
31% done.
32% done.
33% done.
34% done.
35% done.
36% done.
37% done.
38% done.
39% done.
40% done.
41% done.
42% done.
43% done.
44% done.
45% done.
46% done.
47% done.
48% done.
49% done.
50% done.
51% done.
52% done.
53% done.
54% done.
55% done.
56% done.
57% done.
58% done.
59% done.
60% done.
61% done.
62% done.
63% done.
64% done.
65% done.
66% done.
67% done.
68% done.
69% done.
70% done.
71% done.
72% done.
73% done.
74% done.
75% done.
76% done.
77% done.
78% done.
79% done.
80% done.
81% done.
82% done.
83% done.
84% done.
85% done.
86% done.
87% done.
88% done.
89% done.
90% done.
91% done.
92% done.
93% done.
94% done.
95% done.
96% done.
97% done.
98% done.
99% done.
100% done.


In [13]:
Hematocrit = makeset(Hematocrit, 42)
HematocritSave = makechannel(Hematocrit, 'Hematocrit')
HematocritSave.to_csv(r'/Users/alex/LOSDataHematocrit.csv')

1% done.
2% done.
3% done.
4% done.
5% done.
6% done.
7% done.
8% done.
9% done.
10% done.
11% done.
12% done.
13% done.
14% done.
15% done.
16% done.
17% done.
18% done.
19% done.
20% done.
21% done.
22% done.
23% done.
24% done.
25% done.
26% done.
27% done.
28% done.
29% done.
30% done.
31% done.
32% done.
33% done.
34% done.
35% done.
36% done.
37% done.
38% done.
39% done.
40% done.
41% done.
42% done.
43% done.
44% done.
45% done.
46% done.
47% done.
48% done.
49% done.
50% done.
51% done.
52% done.
53% done.
54% done.
55% done.
56% done.
57% done.
58% done.
59% done.
60% done.
61% done.
62% done.
63% done.
64% done.
65% done.
66% done.
67% done.
68% done.
69% done.
70% done.
71% done.
72% done.
73% done.
74% done.
75% done.
76% done.
77% done.
78% done.
79% done.
80% done.
81% done.
82% done.
83% done.
84% done.
85% done.
86% done.
87% done.
88% done.
89% done.
90% done.
91% done.
92% done.
93% done.
94% done.
95% done.
96% done.
97% done.
98% done.
99% done.
100% done.


In [16]:
pH = makeset(pH, 7.4)
pHSave = makechannel(pH, 'pH')
pHSave.to_csv(r'/Users/alex/LOSDatapH.csv')

1% done.
2% done.
3% done.
4% done.
5% done.
6% done.
7% done.
8% done.
9% done.
10% done.
11% done.
12% done.
13% done.
14% done.
15% done.
16% done.
17% done.
18% done.
19% done.
20% done.
21% done.
22% done.
23% done.
24% done.
25% done.
26% done.
27% done.
28% done.
29% done.
30% done.
31% done.
32% done.
33% done.
34% done.
35% done.
36% done.
37% done.
38% done.
39% done.
40% done.
41% done.
42% done.
43% done.
44% done.
45% done.
46% done.
47% done.
48% done.
49% done.
50% done.
51% done.
52% done.
53% done.
54% done.
55% done.
56% done.
57% done.
58% done.
59% done.
60% done.
61% done.
62% done.
63% done.
64% done.
65% done.
66% done.
67% done.
68% done.
69% done.
70% done.
71% done.
72% done.
73% done.
74% done.
75% done.
76% done.
77% done.
78% done.
79% done.
80% done.
81% done.
82% done.
83% done.
84% done.
85% done.
86% done.
87% done.
88% done.
89% done.
90% done.
91% done.
92% done.
93% done.
94% done.
95% done.
96% done.
97% done.
98% done.
99% done.
100% done.


In [17]:
OxySat = makeset(OxySat, 97.5)
OxySatSave = makechannel(OxySat, 'OxySat')
OxySatSave.to_csv(r'/Users/alex/LOSDataOxySat.csv')

1% done.
2% done.
3% done.
4% done.
5% done.
6% done.
7% done.
8% done.
9% done.
10% done.
11% done.
12% done.
13% done.
14% done.
15% done.
16% done.
17% done.
18% done.
19% done.
20% done.
21% done.
22% done.
23% done.
24% done.
25% done.
26% done.
27% done.
28% done.
29% done.
30% done.
31% done.
32% done.
33% done.
34% done.
35% done.
36% done.
37% done.
38% done.
39% done.
40% done.
41% done.
42% done.
43% done.
44% done.
45% done.
46% done.
47% done.
48% done.
49% done.
50% done.
51% done.
52% done.
53% done.
54% done.
55% done.
56% done.
57% done.
58% done.
59% done.
60% done.
61% done.
62% done.
63% done.
64% done.
65% done.
66% done.
67% done.
68% done.
69% done.
70% done.
71% done.
72% done.
73% done.
74% done.
75% done.
76% done.
77% done.
78% done.
79% done.
80% done.
81% done.
82% done.
83% done.
84% done.
85% done.
86% done.
87% done.
88% done.
89% done.
90% done.
91% done.
92% done.
93% done.
94% done.
95% done.
96% done.
97% done.
98% done.
99% done.
100% done.


In [18]:
ResRate = makeset(ResRate, 16)
ResRateSave = makechannel(ResRate, 'ResRate')
ResRateSave.to_csv(r'/Users/alex/LOSDataResRate.csv')

1% done.
2% done.
3% done.
4% done.
5% done.
6% done.
7% done.
8% done.
9% done.
10% done.
11% done.
12% done.
13% done.
14% done.
15% done.
16% done.
17% done.
18% done.
19% done.
20% done.
21% done.
22% done.
23% done.
24% done.
25% done.
26% done.
27% done.
28% done.
29% done.
30% done.
31% done.
32% done.
33% done.
34% done.
35% done.
36% done.
37% done.
38% done.
39% done.
40% done.
41% done.
42% done.
43% done.
44% done.
45% done.
46% done.
47% done.
48% done.
49% done.
50% done.
51% done.
52% done.
53% done.
54% done.
55% done.
56% done.
57% done.
58% done.
59% done.
60% done.
61% done.
62% done.
63% done.
64% done.
65% done.
66% done.
67% done.
68% done.
69% done.
70% done.
71% done.
72% done.
73% done.
74% done.
75% done.
76% done.
77% done.
78% done.
79% done.
80% done.
81% done.
82% done.
83% done.
84% done.
85% done.
86% done.
87% done.
88% done.
89% done.
90% done.
91% done.
92% done.
93% done.
94% done.
95% done.
96% done.
97% done.
98% done.
99% done.
100% done.


In [19]:
Platelets = makeset(Platelets, 240)
PlateletsSave = makechannel(Platelets, 'Platelets')


1% done.
2% done.
3% done.
4% done.
5% done.
6% done.
7% done.
8% done.
9% done.
10% done.
11% done.
12% done.
13% done.
14% done.
15% done.
16% done.
17% done.
18% done.
19% done.
20% done.
21% done.
22% done.
23% done.
24% done.
25% done.
26% done.
27% done.
28% done.
29% done.
30% done.
31% done.
32% done.
33% done.
34% done.
35% done.
36% done.
37% done.
38% done.
39% done.
40% done.
41% done.
42% done.
43% done.
44% done.
45% done.
46% done.
47% done.
48% done.
49% done.
50% done.
51% done.
52% done.
53% done.
54% done.
55% done.
56% done.
57% done.
58% done.
59% done.
60% done.
61% done.
62% done.
63% done.
64% done.
65% done.
66% done.
67% done.
68% done.
69% done.
70% done.
71% done.
72% done.
73% done.
74% done.
75% done.
76% done.
77% done.
78% done.
79% done.
80% done.
81% done.
82% done.
83% done.
84% done.
85% done.
86% done.
87% done.
88% done.
89% done.
90% done.
91% done.
92% done.
93% done.
94% done.
95% done.
96% done.
97% done.
98% done.
99% done.
100% done.


FileNotFoundError: [Errno 2] No such file or directory: '/cs/home/psxau1/db/LOSDataPlatelets.csv'

In [20]:
PlateletsSave.to_csv(r'LOSDataPlatelets.csv')

In [21]:
Weight = makeset(Weight, 87)
WeightSave = makechannel(Weight, 'Weight')
WeightSave.to_csv(r'LOSDataWeight.csv')

1% done.
2% done.
3% done.
4% done.
5% done.
6% done.
7% done.
8% done.
9% done.
10% done.
11% done.
12% done.
13% done.
14% done.
15% done.
16% done.
17% done.
18% done.
19% done.
20% done.
21% done.
22% done.
23% done.
24% done.
25% done.
26% done.
27% done.
28% done.
29% done.
30% done.
31% done.
32% done.
33% done.
34% done.
35% done.
36% done.
37% done.
38% done.
39% done.
40% done.
41% done.
42% done.
43% done.
44% done.
45% done.
46% done.
47% done.
48% done.
49% done.
50% done.
51% done.
52% done.
53% done.
54% done.
55% done.
56% done.
57% done.
58% done.
59% done.
60% done.
61% done.
62% done.
63% done.
64% done.
65% done.
66% done.
67% done.
68% done.
69% done.
70% done.
71% done.
72% done.
73% done.
74% done.
75% done.
76% done.
77% done.
78% done.
79% done.
80% done.
81% done.
82% done.
83% done.
84% done.
85% done.
86% done.
87% done.
88% done.
89% done.
90% done.
91% done.
92% done.
93% done.
94% done.
95% done.
96% done.
97% done.
98% done.
99% done.
100% done.


FileNotFoundError: [Errno 2] No such file or directory: '/cs/home/psxau1/db/LOSDataWeight.csv'

In [22]:
WeightSave.to_csv(r'LOSDataWeight.csv')